In [5]:
import os
import czifile
import tifffile
from tqdm import tqdm
from PIL import Image

Image.MAX_IMAGE_PIXELS = None

# Path to directory containing the CZI files
czi_dir = 'czi_files'

# Directory to save the TIFF files
# creates subfolder within directory 'TIFF files'
tiff_dir = os.path.join(czi_dir, 'tiff_files')
os.makedirs(tiff_dir, exist_ok=True)

max_width = 10000  # Maximum width after downsampling
max_height = 10000  # Maximum height after downsampling
max_file_size = 3300000000 # set max file size at 1.1GB

# Get all files in directory ending with '.czi'
czi_files = [file for file in os.listdir(czi_dir) if file.endswith('.czi')]


# Loop over all files in the directory
for filename in tqdm(czi_files, desc="Converting files"):
    # Construct the full file path
    czi_file = os.path.join(czi_dir, filename)

    # check to see if file size < 1GB
    file_size = os.path.getsize(czi_file)

    # if file is small enough, proceed with conversion and downsampling
    if file_size < max_file_size:

        try:
            # Try to read the CZI file
            with czifile.CziFile(czi_file) as czi:
                img_data = czi.asarray()

        except Exception as e:
            print(f"File {filename} raised an error during processing: {str(e)}. Continuing with next file.")        
            continue
            
        # Construct the output file path (change the extension to .tiff)
        tiff_file = os.path.join(tiff_dir, os.path.splitext(filename)[0] + '.tiff')
        
        # Save as a TIFF file
        tifffile.imsave(tiff_file, img_data)

        try:
        #downsample and save over original tiff file
            with Image.open(tiff_file) as img:
                # Resize the image while preserving aspect ratio
                img.thumbnail((max_width, max_height))
                # Save the resized image
                img.save(tiff_file)
            
            print(f"Successfully downsampled {filename}.")
        
        except Exception as e:
            print(f"An error occurred while processing {filename}: {e}. Continuing with next file.")
    
    # if file is too large, skip.
    else:
        print(f"{filename} exceeds max file size. Skipping.")

print("All CZI files have been converted to TIFF and downsampled.")


Converting files:   0%|          | 0/14 [00:00<?, ?it/s]/var/folders/9w/r3_6vknj0ys3b0ndnws_3l5m0000gr/T/ipykernel_85027/2762794849.py:49: DeprecationWarning: <tifffile.imsave> is deprecated. Use tifffile.imwrite
  tifffile.imsave(tiff_file, img_data)
Converting files:   7%|▋         | 1/14 [01:52<24:17, 112.14s/it]

Successfully downsampled G23-3_C23_R-3a-2023_10_09__0012.czi.


Converting files:  14%|█▍        | 2/14 [04:04<24:45, 123.76s/it]

Successfully downsampled G23-3_C23_L-3a-2023_10_09__0010.czi.


Converting files:  21%|██▏       | 3/14 [05:50<21:12, 115.69s/it]

...

Successfully downsampled G23-3_C23_L-4a-2023_10_09__0011.czi.


Converting files:  93%|█████████▎| 13/14 [24:35<01:58, 118.42s/it]

Successfully downsampled G23-3_C23_R-4a-2023_10_09__0013.czi.


Converting files: 100%|██████████| 14/14 [26:33<00:00, 113.79s/it]

Successfully downsampled G23-3_C23_R_2a_2023_08_24__0005.czi.
All CZI files have been converted to TIFF and downsampled.
